In [1]:
print("hello")

hello


# Homework 1

## Problem 1

Using introspection to inform AI in modeling human cognitive processes is an obvious first stage of modeling human thought. However, the effectiveness of this strategy depends both on the goal of the AI. If the goal is to create a computer that thinks in a similar manner to a human (that is, approaching problems in the same way), then introspection probably makes sense. In this case, it isn't so much the results that matter, as the way the program gets there. If the goal is a program that ouptuts similar results to a human's conclusion, then I would say that introspection will probably not be the best way to do that. People's minds are complex, even at the level at which we can examine our own minds. The way we think is not the most efficient because of this complexity. Because of this, even if we want human-style conclusions, I would say that introspection and modeling the computations after human thought would not be the best way to get there.

The goal could also be finding accurate results regardless of how a person would approach the problem or if they would come up with an accurate answer. This is a different question than what was asked, but I would say it is a more important question. If the computers can do things that we can't, then they contribute much more value.

Revisiting the issue of modeling patterns of human thought processes, introspection would solve the issue at one level. Neural networks would solve the issue on another level. Introspection, though, would approach the problem in the same way as a person. It would value the same things, and it would start with the same focuses. If we want the AI program to focus on the same kinds of patterns and connections as a person would, then introspection is the only way to do it. Anything else either surpasses this level or is too basic for us to be able to model the whole human mind and make it to this level. 

## Problem 2

In [ ]:
import string, random
import numpy as np
from search import Problem, hill_climbing, simulated_annealing

class TSP(Problem):
    def __init__(self, initial, distances, num_cities):
        self.initial = initial
        self.distances = distances
        self.num_cities = num_cities

    def actions(self, state):
        # lists all pairs of cities that could be flipped in the order
        starts = np.zeros(0, dtype=np.int8)
        ends   = np.zeros(0, dtype=np.int8)

        for city in range(self.num_cities):
            # create list of this city as the first city to flip
            starts = np.append(starts, np.full(self.num_cities-city-1, city, dtype=np.int8))
            # create list of all the later cities as second cities to switch (all earlier cities will have the current city as the second city of the flip)
            ends   = np.append(ends, np.arange(city+1, self.num_cities, dtype=np.int8))

        actions = np.append(starts, ends)
        actions = actions.reshape(-1, 2, order='F')
        return actions

    def result(self, state, action):
        # return the state that results from executing the given action in the current state
        new_state = np.copy(state)
        city1 = state[action[0]]
        city2 = state[action[1]]
        new_state[action[0]] = city2
        new_state[action[1]] = city1
        return new_state

    def value(self, state):
        value = 0
        for i in range(len(state)):
            value += self.distances[state[i]][state[(i+1)%self.num_cities]]
        return -value

### State and action representations
For the local search, the TSP extends the Problem from csp.py and implements states and actions.
The states, as in the n-queens problem, are a complete solution. In this case, we start with a list of the cities. The cities are numbered, so they are in that order to start.
The order of the cities in the list represents the state. The last city loops around and leads back to the first city. This first city is only once in the list because the value() considers that it needs to loop around to the beginning again.
The actions are represented as pairs of cities. Any two cities can be swapped in the travel order. Any of these actions keep the state as a complete solution, but it may make the value larger or smaller.

### TSP with any number of cities
Below, a system with any number of cities can be created by changing the num_cities variable. From there, it randomly calculates distances between all the cities.
The salesman can travel from any city to any other city in one step.

### Hill-climbing vs Simulated Annealing
The hill-climbing algorithm works consistently better and by a significant amount, regardless of the size of the system. (As long as the number of cities is high enough that both algorithms do not always get the ideal solution, as happens with num_cities < 5.)
If the cities were geographically coherent, then this would make sense. Since they are not, this is slightly more surprising.
I would have expected that simulated annealing would work better here. For this problem (since the cities are not geogrpahically cohert), there are times that it is beneficial to be willing to jump to a worse value in order to get out of a local optima and find the global optima.

In [ ]:
# set up city with random but symmetrical distances
num_cities = 10
min_dist = 1
max_dist = 10
initial = np.arange(num_cities)

distances = np.zeros((num_cities, num_cities))
for i in range(num_cities):
    for j in range(num_cities-i):
        row = i
        col = num_cities-j-1
        if row == (col):
            distances[row][col] = 0
        else:
            distances[row][col] = random.randint(min_dist, max_dist)
            distances[col][row] = distances[row][col]

print("Distances:")
print(distances)

p = TSP(initial, distances, num_cities)
print("Initial:")
print(initial)

hill_solution = hill_climbing(p)
print("\nHill-climbing solution:")
print(hill_solution)
print("Hill-climbing value = " + str(p.value(hill_solution)))

annealing_solution = simulated_annealing(p)
print("\nSimulated Annealing soltion:")
print(annealing_solution)
print("Simulated Annealing value = " + str(p.value(annealing_solution)))

## Problem 3

In [ ]:
from csp import min_conflicts, CSP, parse_neighbors


# constants for keeping track of order of faculty, time and room in variable lists
PROF = 0
TIME = 1
ROOM = 2

courses = ['cs108', 'cs112', 'cs212', 'cs214', 'cs232', 'cs262', 'cs300']
faculty = ['dschuurman', 'adams', 'vlinden', 'hplantin', 'pmb4']
times = ['mwf900', 'mwf1030', 'mwf1130', 'mwf1230', 'mwf130']
rooms = ['nh253', 'sb382']
assignments = {'cs108': 'dschuurman', 'cs112': 'adams', 'cs212':'hplantin', 'cs214':'adams', 'cs232':'pmb4', 'cs262':'vlinden', 'cs300':'vlinden'}
num_courses = len(courses)

# neighbors: list all pairs of courses in the format needed for parse_neighbors
# does not include pairing courses with themselves or pairs that are the same in the other order
neighbors_string = ""
for i in range(num_courses):
    for j in range(num_courses-i-1):
        course1 = courses[i]
        course2 = courses[num_courses-j-1]
        neighbors_string += course1 + ":" + course2 + ";"

neighbors_string = neighbors_string[:-1] # get rid of the last semicolon
neighbors = parse_neighbors(neighbors_string, variables=courses)

# list all teaching combinations (of faculty, time, and room) that could be assigned to a course
domains = {}
for course in courses:
    domains[course] = []
    for prof in faculty:
        for time in times:
            for room in rooms:
                domains[course].append( [prof, time, room] )


# return whether the pair of courses do not conflict
def scheduling_constraints(courseA, variableA, courseB, variableB):
    """
    if the courses have the same room, same time, that fails the requirements
    if the courses have the same prof, same time, that fails the requirements
    if the courses have the wrong prof, that also fails the requirements
    """

    # check if they are in the same room or with the same faculty member at the same time
    if variableA[TIME] == variableB[TIME] and (variableA[ROOM] == variableB[ROOM] or variableA[PROF] == variableB[PROF]):
        return False

    # check that each course is with the correct professor
    if variableA[PROF] != assignments[courseA] or variableB[PROF] != assignments[courseB]:
        return False

    # if it didn't fail yet, it's valid
    return True


result = min_conflicts(CSP(courses, domains, neighbors, scheduling_constraints))
print(result)

An explanation of the reasoning of this solution should start at the end. Because of the requirements, the min_conflicts function and the CSP class are necessary components of the solution. To decide how to go about filling the requirements of the CSP class constructor, I looked at the Sudoku, Queens, and Zebra problems. The Sudoku was a particularly helpful one. The Zebra problem wsa confusing because of documentation issues, and the queens problem sent me down the wrong track with building a subclass of CSP.
Making the list of requirements was very straightforward from the problem description.
From there, the problem got more complicated to approach. It make sense to have the courses be the variables because they are fixed and the other parameters need to be assigned to them.
Once we were told that it worked to make all the combinations of faculty, time, and room domains, the rest of it could use strategies from the Zebra problem, but actually simpler.

As you can see by running the code, this solution works and gives a schedule that fills the requirements.